# Creating a dispersed axon bundle representation

In [1]:
# load the necessary modules
from mipy.data import saved_acquisition_schemes
acq_scheme = saved_acquisition_schemes.wu_minn_hcp_acquisition_scheme()

In Microstructure Imaging, the brain's axon bundles are represented as having both an intra- and an extra-axonal compartment. Intra-axonal compartments are typically represented using cylinder models, while extra-axonal compartments are typically Gaussian. To simulate the phenomenon of axon dispersion, it is important to see a bundle as a single "micro-environment".

Mipy allows to create such a "bundle representation" using distribute_models module. A distributed model can be seen as a type of sub-multi-compartment model that can contain multiple single models representations (Sticks, Cylinders, Zeppelin). The contained single models all experience the same distribution parameters. We can choose spherical distributions such as the Watson or Bingham, or spatial distributions such as the Gamma for models such as AxCaliber.

In this example, we will start by creating a Watson-dispersed Stick and Zeppelin representation.

In [2]:
from mipy.signal_models import gaussian_models, cylinder_models
from mipy.distributions import distribute_models

stick = cylinder_models.C1Stick()
zeppelin = gaussian_models.G4Zeppelin()
watson_bundle = distribute_models.SD1WatsonDistributed(models=[stick, zeppelin])

We can list the parameters of this representation as follows:

In [3]:
watson_bundle.parameter_names

['SD1Watson_1_odi',
 'G4Zeppelin_1_lambda_perp',
 'SD1Watson_1_mu',
 'C1Stick_1_lambda_par',
 'G4Zeppelin_1_lambda_par',
 'partial_volume_0']

The parameters 'SD1Watson_1_odi' and 'SD1Watson_1_mu' represent the Orientation Dispersion Index (ODI) and orientation of the Watson distribution. The others are parameters of the Stick and Zeppelin models.

Notice is that there is only one "mu" parameter, as the orientation of the Stick and Zeppelin are now tied to that of the Watson distribution, i.e. the Stick and Zeppelin are now always aligned.

Lastly, the 'partial_volume_0' parameter represents the volume fraction of the first input model - in our case the stick - where the second volume fraction is defined as partial_volume_1 = 1-partial_volume_0.

# Imposing parameter links: tortuous, equal and fixed parameters

Often, Microstructure models impose parameter constraints in their model definition. In NODDI's case, they impose axon tortuosity and parameter equality:
- Certain parameters are the same between the zeppelin and the stick, such as the parallel diffusivity $\lambda_\parallel$.
- The model imposes a Tortuosity constraint, such that $\lambda_\perp=(1-vf)\lambda_\parallel$.

Mipy allows to straightforwardly impose such constraints on a microstructure model. First, we set the tortuosity constraint using the set_tortuous_parameter function, and inputting the correct names for the $\lambda_\perp$, vf and $\lambda_\parallel$ parameters. 

In [4]:
watson_bundle.set_tortuous_parameter('G4Zeppelin_1_lambda_perp', 'G4Zeppelin_1_lambda_par', 'partial_volume_0')

It can be seen that $\lambda_\perp$ is now removed from the list of parameters, as it has been made dependent on $\lambda_\parallel$ and $vf$.

In [5]:
watson_bundle.parameter_names

['SD1Watson_1_odi',
 'SD1Watson_1_mu',
 'C1Stick_1_lambda_par',
 'G4Zeppelin_1_lambda_par',
 'partial_volume_0']

To set the parallel diffusivities of the Stick and Zeppelin equal, we use the set_equal_parameter function

In [6]:
watson_bundle.set_equal_parameter('C1Stick_1_lambda_par', 'G4Zeppelin_1_lambda_par')

Notice how the Zeppelin's $\lambda_\parallel$ was now removed.

In [7]:
watson_bundle.parameter_names

['SD1Watson_1_odi',
 'SD1Watson_1_mu',
 'C1Stick_1_lambda_par',
 'partial_volume_0']

Finally, many Microstructure Models will somehow fix its diffusivities to avoid fitting them from the data. As an example, we will finally fix the Stick's $\lambda_\parallel$ to a typical value that is used for intra-axonal diffusivity

In [8]:
watson_bundle.set_fixed_parameter('C1Stick_1_lambda_par', 1.7e-9)

Which leaves us with 3 parameters left.

In [9]:
watson_bundle.parameter_names

['SD1Watson_1_odi', 'SD1Watson_1_mu', 'partial_volume_0']

The watson_bundle can finally be used in a MultiCompartment model like any other, after which it can be used for simulating and fitting data.

In [10]:
from mipy.core.modeling_framework import MultiCompartmentModel
mcmodel = MultiCompartmentModel(models=[watson_bundle])
mcmodel.parameter_cardinality

OrderedDict([('SD1WatsonDistributed_1_SD1Watson_1_mu', 2),
             ('SD1WatsonDistributed_1_partial_volume_0', 1),
             ('SD1WatsonDistributed_1_SD1Watson_1_odi', 1)])

Once the multi-compartment model is made, simulating and fitting data is exactly as in the previous tutorials.